# 日本語Word2Vecサンプル

## Step1 青空文庫より「三四郎」をダウンロードし整形するまで

In [1]:
# ファイルダウンロード、解凍
!wget http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
!unzip 794_ruby_4237.zip
!ls -l sanshiro.txt

--2018-02-12 03:51:24--  http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
Resolving www.aozora.gr.jp (www.aozora.gr.jp)... 59.106.13.115
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158711 (155K) [application/zip]
Saving to: ‘794_ruby_4237.zip’

100%[======================================>] 158,711      234KB/s   in 0.7s   

2018-02-12 03:51:25 (234 KB/s) - ‘794_ruby_4237.zip’ saved [158711/158711]

Archive:  794_ruby_4237.zip
  inflating: sanshiro.txt            
-rw------- 1 s496-58045080f3eb61-3a788f3af7c0 users 363871 Jun 19  2014 sanshiro.txt


In [2]:
# ファイル読込み、内部表現化
f = open('sanshiro.txt')
text_sjis = f.read()
f.close()
text = text_sjis.decode('sjis')

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split(u'\-{5,}',text)[2]
# フッタ部分の除去
text = re.split(u'底本：',text)[0]
# | の除去
text = text.replace(u'|', u'')
# ルビの削除
text = re.sub(u'《.+?》', u'', text)
# 入力注の削除
text = re.sub(u'［＃.+?］', u'',text)
# 空行の削除
text = re.sub(u'\n\n', '\n', text) 
text = re.sub(u'\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print text[:100]
# 見やすくするため、空行 
print
print
# 後ろの100文字の表示 
print text[-100:]



一

　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌を


取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。






## Step2 Janomeを使い三四郎テキストから単語リストを生成

In [3]:
# Janomeのインストール
!pip install janome

# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in[u'名詞', u'動詞']]

#  関数テスト
ret = extract_words(u'三四郎は京都でちょっと用があって降りたついでに。')
for word in ret:
    print word

# 全体のテキストを句点(u'。')で区切った配列にする。 
sentences = text.split(u'。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print word

三四郎
京都
ちょっと
用
ある
降りる
ついで
一
する
目
さめる
女
隣
じいさん
話
始める
いる


## Step 3 準備したデータを用いてWord2Vecのモデル作成、学習実施

In [4]:
# Word2Vecライブラリの導入
!pip install gensim  

# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [5]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print model.__dict__['wv'][u'世間']

[-7.17373252e-01 -1.97198868e-01 -4.05806184e-01  4.78572160e-01
  7.83140659e-02 -2.68161565e-01  2.85706073e-01  2.78858453e-01
 -5.19911349e-01 -5.75572960e-02  6.86857104e-01  4.98317957e-01
  6.05263412e-01  3.16094846e-01 -4.48681228e-02 -1.04482144e-01
 -5.04335165e-01 -1.66607097e-01 -5.66436172e-01 -6.22701883e-01
  1.04123749e-01 -3.26531082e-01  2.04495281e-01  1.17827944e-01
 -6.03273451e-01  3.92535657e-01  2.52224594e-01 -3.01406384e-01
 -9.17062256e-03  5.40924788e-01  3.29021901e-01  1.30421907e-01
  4.34206538e-02  1.05009042e-01  4.90578115e-01  4.25693929e-01
  4.07388598e-01  5.71348257e-02  3.06507081e-01  7.30050385e-01
 -2.92029023e-01 -1.04512274e-03 -5.77341318e-01  3.96862209e-01
 -3.45898002e-01  7.82120705e-01 -3.57666552e-01  7.03914881e-01
  4.62674677e-01  1.78554222e-01  4.13742900e-01  3.66649419e-01
  1.39880091e-01 -5.98248422e-01  7.58831203e-02 -1.64480746e-01
  3.46842974e-01  7.38604963e-01 -2.64371723e-01  1.68999478e-01
  5.82948744e-01  4.90597

In [6]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=[u'世間']) 
for item in ret:
    print item[0], item[1]

聞こえる 0.558363080025
外国 0.550490140915
喝采 0.544633328915
自己 0.491480588913
堪える 0.475788533688
くせ 0.459737330675
有す 0.450292408466
今日 0.446328788996
える 0.44494330883
全体 0.440056592226
